# Install lib

In [86]:
!pip install sentence_transformers

# Import lib

In [87]:
import pandas as pd
from copy import deepcopy
from collections import Counter

In [88]:
import warnings
warnings.filterwarnings('ignore')

# 1. Keyword

In [89]:
keyword = pd.read_excel("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/keyword_from_Mr_Hao.xlsx")
keyword

,Tên biểu đồ,Code chart,Keyword
0,Biểu đồ cảnh báo 2,ALARM_CHART_2,Thông tin chỉ tiêu
1,Biểu đồ RANK,RANK,Xếp hạng
2,Biểu đồ đường,LINE,Xu thế
3,Biểu đồ cột,COLUMN,"Tăng trưởng, hoàn thành"
4,Biểu đồ cơ cấu,PIE,Cơ cấu
5,Biểu đồ kết hợp đường và cột,COMBINE,Tăng giảm
6,Biểu đồ thị phần,TYPE_CHART_TABLE_THI_PHAN,Thị phần


# 2. Service ID information

In [90]:
service_id_all = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/SERVICE_ID_infor.csv")
service_id_list = list(set(service_id_all['SERVICE_ID']))
service_id_dictionary = {u: list(set(service_id_all[service_id_all['SERVICE_ID']==u]['SERVICE_NAME']))[0] for u in service_id_list}

# 3. Cat Item

In [91]:
# Mở file và lấy feature cần thiết Cat Item
cat_item = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/CAT_ITEM_202307280947.csv")
cat_item.drop(columns=["CONTENT", "EDITABLE", "PARENT_ITEM_ID", "CATEGORY_ID", "UPDATE_TIME", "UPDATE_USER",
                       "CATEGORY_CODE"], inplace=True)

In [92]:
cat_item

,ITEM_ID,ITEM_CODE,ITEM_NAME,ITEM_VALUE,POSITION,DESCRIPTION,STATUS
0,990,TYPE_CHART_TABLE,Biểu đồ TABLE,TABLE,17.0,TABLE,1
1,918,TYPE_CHART_LINE_DASHED,LINE_DASHED chart,LINE_DASHED,17.0,Biểu đồ LINE_DASHED,0
2,306083,TYPE_CHART_ALARM_CHART_2,Biểu đồ cảnh báo 2,ALARM_CHART_2,21.0,Biểu đồ cảnh báo 2,1
3,915,TYPE_CHART_INFO_MAIN,Biểu đồ INFO_MAIN,INFO_MAIN,14.0,INFO_MAIN,1
4,916,TYPE_CHART_SHARE_TIME,Biểu đồ cơ cấu cột chồng,SHARE_TIME,15.0,SHARE_TIME,1
5,917,TYPE_CHART_RANK,Biểu đồ RANK,RANK,16.0,RANK,1
6,304828,TYPE_CHART_NESTED_PIE,Biểu đồ NESTED_PIE,NESTED_PIE,20.0,Biểu đồ NESTED_PIE,1
7,304802,TYPE_CHART_BLOCK_INFO,Biểu đồ BLOCK_INFO,BLOCK_INFO,19.0,Biểu đồ BLOCK_INFO,1
8,305027,BLOCK_INFO_2,Biểu đồ BLOCK_INFO_2,BLOCK_INFO_2,19.0,Biểu đồ BLOCK_INFO_2,1
9,914,TYPE_CHART_INFO_OBJ,Biểu đồ INFO_OBJ,INFO_OBJ,13.0,INFO_OBJ,1


# 4. D_CHART_DEPT_SERVICE

## 4.1. Data

Trả về Chart_ID, Type_Chart
* Chart_ID: Song ánh với TITLE_CHART, DEPT_CODE, CYCTIME
* Type_Chart: TYPE_CHART

In [93]:
chart_dept = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/D_CHART_DEPT_SERVICE_202307280943.csv")
chart_dept.drop(columns=["GROUP_CHART"], inplace=True)
chart_dept['SERVICE_NAME'] = chart_dept['SERVICE_ID'].apply(lambda i: service_id_dictionary[i] if i in service_id_dictionary.keys() else "")
chart_dept

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,UPDATE_DATE,SERVICE_NAME
0,77254,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,MINI,NGAY,2022-08-08 09:47:22.000,DOANH THU DỊCH VỤ (chi tiết)
1,77255,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,TITLE_MOBILE,NGAY,2022-08-08 09:47:22.000,DOANH THU DỊCH VỤ (chi tiết)
2,77256,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,CHART_RANK_2,NGAY,2022-08-08 09:47:22.000,DOANH THU DỊCH VỤ (chi tiết)
3,77257,S079,970,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,4G Lũy kế
4,77258,S079,971,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,3G Tăng giảm
...,...,...,...,...,...,...,...,...,...
57329,132568,TD,6003,23456,DANH SÁCH NHIỆM VỤ CHẬM TIẾN ĐỘ,TABLE_CONTENT,NaN,2023-05-25 09:06:41.000,Chậm tiến độ
57330,132353,TD,6002,23457,DANH SÁCH SẮP ĐẾN HẠN,TABLE_CONTENT,NaN,2023-05-25 09:12:43.000,Sắp đến hạn
57331,132354,TD,6003,23456,DANH SÁCH NHIỆM VỤ CHẬM TIẾN ĐỘ,TABLE_CONTENT,NaN,2023-05-25 09:06:41.000,Chậm tiến độ
57332,132355,TD,6002,23457,DANH SÁCH SẮP ĐẾN HẠN,TABLE_CONTENT,NaN,2023-05-25 09:12:43.000,Sắp đến hạn


## 4.2. Analyze

In [96]:
# 17430 cái CHART ID
chart_id_list = chart_dept['CHART_ID'].unique().tolist()
chart_id_list[:5]
len(chart_id_list)

17430

In [97]:
# 47 loại biểu đồ
chart_type_list = chart_dept['TYPE_CHART'].unique().tolist()
chart_type_list[:5]
len(chart_type_list)

47

In [98]:
# 1 chart id có nhiều dept_code
for i in range(len(chart_id_list)):
    if i in []:
        continue
    if len(list(set(chart_dept[chart_dept['CHART_ID']==chart_id_list[i]]['DEPT_CODE'].to_list()))) != 1:
        print(i)
        break

chart_dept[chart_dept['CHART_ID']==chart_id_list[i]]

105


,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,UPDATE_DATE,SERVICE_NAME
275,76664,VTC,806,19079,VTG_TB DI ĐỘNG THỰC 4G_CƠ CẤU ĐƠN VỊ (LK KỆ TH...,SHARE_TIME,THANG,2022-12-13 18:16:32.000,Thuê bao 4G (LK toàn mạng)
276,76665,VTZ,806,19079,VTG_TB DI ĐỘNG THỰC 4G_CƠ CẤU ĐƠN VỊ (LK KỆ TH...,SHARE_TIME,THANG,2022-12-13 18:16:32.000,Thuê bao 4G (LK toàn mạng)
277,76666,VTP,806,19079,VTG_TB DI ĐỘNG THỰC 4G_CƠ CẤU ĐƠN VỊ (LK KỆ TH...,SHARE_TIME,THANG,2022-12-13 18:16:32.000,Thuê bao 4G (LK toàn mạng)
278,76667,VTL,806,19079,VTG_TB DI ĐỘNG THỰC 4G_CƠ CẤU ĐƠN VỊ (LK KỆ TH...,SHARE_TIME,THANG,2022-12-13 18:16:32.000,Thuê bao 4G (LK toàn mạng)
279,76668,MYN,806,19079,VTG_TB DI ĐỘNG THỰC 4G_CƠ CẤU ĐƠN VỊ (LK KỆ TH...,SHARE_TIME,THANG,2022-12-13 18:16:32.000,Thuê bao 4G (LK toàn mạng)
280,76669,MOV,806,19079,VTG_TB DI ĐỘNG THỰC 4G_CƠ CẤU ĐƠN VỊ (LK KỆ TH...,SHARE_TIME,THANG,2022-12-13 18:16:32.000,Thuê bao 4G (LK toàn mạng)
281,76670,NAT,806,19079,VTG_TB DI ĐỘNG THỰC 4G_CƠ CẤU ĐƠN VỊ (LK KỆ TH...,SHARE_TIME,THANG,2022-12-13 18:16:32.000,Thuê bao 4G (LK toàn mạng)
282,76671,STL,806,19079,VTG_TB DI ĐỘNG THỰC 4G_CƠ CẤU ĐƠN VỊ (LK KỆ TH...,SHARE_TIME,THANG,2022-12-13 18:16:32.000,Thuê bao 4G (LK toàn mạng)
283,76672,VBD,806,19079,VTG_TB DI ĐỘNG THỰC 4G_CƠ CẤU ĐƠN VỊ (LK KỆ TH...,SHARE_TIME,THANG,2022-12-13 18:16:32.000,Thuê bao 4G (LK toàn mạng)


In [99]:
# 1 chart id có nhiều service_id
for i in range(len(chart_id_list)):
    if i in []:
        continue
    if len(list(set(chart_dept[chart_dept['CHART_ID']==chart_id_list[i]]['SERVICE_ID'].to_list()))) != 1:
        print(i)
        break

chart_dept[chart_dept['CHART_ID']==chart_id_list[i]]

1


,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,UPDATE_DATE,SERVICE_NAME
3,77257,S079,970,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,4G Lũy kế
4,77258,S079,971,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,3G Tăng giảm
5,77259,S079,178,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,3G Lũy kế
6,77260,S079,179,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,3G Tăng/giảm
7,77261,S079,176,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,2G Lũy kế
8,77262,S079,177,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,2021-10-28 16:34:52.000,2G Tăng/giảm


In [100]:
# Chỉ 4 chart_id là có 2 title chart (Chủ yếu là error) [13176, 14156, 16382, 16501], còn lại 1 chart id chỉ có 1 title chart
for i in range(len(chart_id_list)):
    if i in [13176, 14156, 16382, 16501]:
        continue
    if len(list(set(chart_dept[chart_dept['CHART_ID']==chart_id_list[i]]['TITLE_CHART'].to_list()))) != 1:
        print(i)
        break

In [101]:
# 1 chart id chỉ có 1 cyctime
for i in range(len(chart_id_list)):
    if i in []:
        continue
    if len(list(set(chart_dept[chart_dept['CHART_ID']==chart_id_list[i]]['CYCTIME'].to_list()))) != 1:
        print(i)
        break

chart_dept[chart_dept['CHART_ID']==chart_id_list[i]]

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,UPDATE_DATE,SERVICE_NAME
57293,132701,BC,1300206,25558,HOẠT ĐỘNG THU-PHÁT BƯU GỬI LŨY KẾ THÁNG Copy(2...,GROUP_BAR,NGAY,2023-05-30 08:50:03.000,Tỷ lệ thu thành công ngày
57294,132702,BC,1300209,25558,HOẠT ĐỘNG THU-PHÁT BƯU GỬI LŨY KẾ THÁNG Copy(2...,GROUP_BAR,NGAY,2023-05-30 08:50:03.000,Tỷ lệ thu thành công đúng giờ ngày
57295,132703,BC,1500209,25558,HOẠT ĐỘNG THU-PHÁT BƯU GỬI LŨY KẾ THÁNG Copy(2...,GROUP_BAR,NGAY,2023-05-30 08:50:03.000,Tỷ lệ thu thành công đúng giờ ngày


In [102]:
# Có 9512 title 
title_list = chart_dept['TITLE_CHART'].unique().tolist()
title_list[:5]
len(title_list)

9512

In [103]:
# Một title chart chỉ có 1 chart_id
for i in range(len(title_list)):
    if len(list(set(chart_dept[chart_dept['TITLE_CHART']==title_list[i]]['TITLE_CHART'].to_list()))) != 1:
        print(i)
        break

In [104]:
# Check copy in title: Có 88 cái tiêu đề copy
title_list = chart_dept['TITLE_CHART'].unique().tolist()
copy_list = []
for i, title in enumerate(title_list):
    if "Copy" in title:
        copy_list.append(i)

len(copy_list)

88

## 4.3. Tên chỉ tiêu

In [105]:
# Number of service id
serviceid_list = chart_dept['SERVICE_ID'].unique()
len(serviceid_list)

1173

In [106]:
chart_dept[chart_dept['SERVICE_ID']==143]

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,UPDATE_DATE,SERVICE_NAME
0,77254,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,MINI,NGAY,2022-08-08 09:47:22.000,DOANH THU DỊCH VỤ (chi tiết)
1,77255,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,TITLE_MOBILE,NGAY,2022-08-08 09:47:22.000,DOANH THU DỊCH VỤ (chi tiết)
2,77256,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,CHART_RANK_2,NGAY,2022-08-08 09:47:22.000,DOANH THU DỊCH VỤ (chi tiết)
26,77280,G059,143,13169,GLI_DTDV_TỪNG QUÝ,LINE,QUY,2022-06-03 14:00:16.000,DOANH THU DỊCH VỤ (chi tiết)
32,77286,H018,143,12866,HBH_DTDV_TỪNG QUÝ,LINE,QUY,2021-10-26 13:40:09.000,DOANH THU DỊCH VỤ (chi tiết)
...,...,...,...,...,...,...,...,...,...
56331,133164,VDS,143,25613,DOANH THU DỊCH VỤ,MINI,THANG,2023-06-20 10:56:28.000,DOANH THU DỊCH VỤ (chi tiết)
56332,133165,VDS,143,25613,DOANH THU DỊCH VỤ,TITLE_MOBILE,THANG,2023-06-20 10:56:28.000,DOANH THU DỊCH VỤ (chi tiết)
56348,133181,K058,143,25678,KHA_DTDV_TĂNG TRƯỞNG THÁNG,COLUMN,THANG,2023-06-20 16:47:30.000,DOANH THU DỊCH VỤ (chi tiết)
56943,132781,VDS,143,25563,Doanh thu dịch vụ,BAR_ALARM,THANG,2023-05-31 13:32:40.000,DOANH THU DỊCH VỤ (chi tiết)


In [107]:
chart_dept['DATE'] = chart_dept['UPDATE_DATE'].apply(lambda x: x.split()[0])
chart_dept.drop(columns=["UPDATE_DATE"], inplace=True)
chart_dept.head()

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DATE
0,77254,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,MINI,NGAY,DOANH THU DỊCH VỤ (chi tiết),2022-08-08
1,77255,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,TITLE_MOBILE,NGAY,DOANH THU DỊCH VỤ (chi tiết),2022-08-08
2,77256,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,CHART_RANK_2,NGAY,DOANH THU DỊCH VỤ (chi tiết),2022-08-08
3,77257,S079,970,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,4G Lũy kế,2021-10-28
4,77258,S079,971,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,LINE,THANG,3G Tăng giảm,2021-10-28


In [108]:
chart_dept_for_title = deepcopy(chart_dept[['DEPT_CODE', 'SERVICE_ID', 'CHART_ID', 'TITLE_CHART', 'CYCTIME', 'DATE']])
chart_dept_for_title['feature_num'] = chart_dept_for_title['TITLE_CHART'].apply(lambda x: len(x.split("_")))
chart_dept_for_title.head()


,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,CYCTIME,DATE,feature_num
0,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,NGAY,2022-08-08,1
1,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,NGAY,2022-08-08,1
2,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,NGAY,2022-08-08,1
3,S079,970,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,THANG,2021-10-28,3
4,S079,971,12386,STG_TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ TH...,THANG,2021-10-28,3


In [109]:
chart_dept_for_title[chart_dept_for_title['feature_num']==4]

,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,CYCTIME,DATE,feature_num
7717,CT,8550,20970,XU HUONG_DOANH THU XAY LAP_VCC_10 NGAY,NGAY,2022-06-20,4
7718,CT,8550,20970,XU HUONG_DOANH THU XAY LAP_VCC_10 NGAY,NGAY,2022-06-20,4
7719,CT,8575,20973,XU HUONG_DOANH THU NANG LUONG VA CO DIEN_VCC_1...,NGAY,2022-06-20,4
7720,CT,8575,20973,XU HUONG_DOANH THU NANG LUONG VA CO DIEN_VCC_1...,NGAY,2022-06-20,4
7948,VTTNEWNEW,20,20954,XU HUONG_DOANH THU DI DONG_VTT_10 NGAY,NGAY,2022-06-20,4
...,...,...,...,...,...,...,...
56064,VTTNEWNEW,1475,25564,VTT_TD THỰC DV DI ĐỘNG_THEO HƯỚNG_TỪNG QUÝ,QUY,2023-06-20,4
56065,VTTNEWNEW,1486,25571,VTT_TD THỰC DV DI ĐỘNG_THEO HƯỚNG_TỪNG QUÝ,QUY,2023-06-20,4
56066,VTTNEWNEW,1487,25571,VTT_TD THỰC DV DI ĐỘNG_THEO HƯỚNG_TỪNG QUÝ,QUY,2023-06-20,4
56067,VTTNEWNEW,1488,25571,VTT_TD THỰC DV DI ĐỘNG_THEO HƯỚNG_TỪNG QUÝ,QUY,2023-06-20,4


In [110]:
chart_dept_for_title[chart_dept_for_title['SERVICE_ID']==serviceid_list[16]]

,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,CYCTIME,DATE,feature_num
42,H018,157,12620,HBH_DTDV nhóm VTT_cơ cấu dịch vụ,THANG,2022-09-08,3
101,D500,157,12763,DLK_DTDV CỐ ĐỊNH_TỪNG QUÝ,QUY,2022-01-13,3
270,VTTNEWNEW,157,18833,VIETTEL TỈNH TP_DTDV CỐ ĐỊNH_LŨY KẾ NGÀY,NGAY,2021-12-30,3
271,VTTNEWNEW,157,18833,VIETTEL TỈNH TP_DTDV CỐ ĐỊNH_LŨY KẾ NGÀY,NGAY,2021-12-30,3
487,G059,157,13159,GLI_DTDV nhóm VTT_cơ cấu dịch vụ,THANG,2022-09-08,3
...,...,...,...,...,...,...,...
49101,D061,157,16136,DT DỊCH VỤ CỐ ĐỊNH BĂNG RỘNG,THANG,2022-11-16,1
49102,D061,157,16136,DT DỊCH VỤ CỐ ĐỊNH BĂNG RỘNG,THANG,2022-11-16,1
49103,D061,157,16136,DT DỊCH VỤ CỐ ĐỊNH BĂNG RỘNG,THANG,2022-11-16,1
49864,MOV,157,21124,XU HUONG_DOANH THU CO DINH_MOV_10 NGAY,NGAY,2022-06-30,4


In [111]:
title_chart = chart_dept['TITLE_CHART'].unique().tolist()
title_chart = [i.split("_") for i in title_chart]
feature_nums = []
for row in title_chart:
    feature_nums.append(len(row))
    if feature_nums[-1]==4:
        print(row)
print(Counter(feature_nums))
for i in range(len(title_chart)):
    if len(title_chart[i]) < 4:
        for _ in range(4-len(title_chart[i])):
            title_chart[i].append('')

['XU HUONG', 'DOANH THU XAY LAP', 'VCC', '10 NGAY']
['XU HUONG', 'DOANH THU NANG LUONG VA CO DIEN', 'VCC', '10 NGAY']
['XU HUONG', 'DOANH THU DI DONG', 'VTT', '10 NGAY']
['XU HUONG', 'DOANH THU QUOC TE', 'VTT', '10 NGAY']
['XU HUONG', 'DOANH THU BAN THE CAO NOI MANG', 'VDS', '10 NGAY']
['XU HUONG', 'DOANH THU CO DINH', 'VTT', '10 NGAY']
['XU HUONG', 'DICH VU KHAC', 'VDS', '10 NGAY']
['XU HUONG', 'TONG DOANH THU', 'VCC', '10 NGAY']
['XU HUONG', 'DOANH THU GIAI PHAP TICH HOP (B2C)', 'VCC', '10 NGAY']
['XU HUONG', 'DOANH THU HA TANG CHO THUE', 'VCC', '10 NGAY']
['XU HUONG', 'TB THỰC', 'VTC', '10 NGÀY']
['XU HUONG', 'DT DICH VU MOI', 'VTT', '10 NGAY']
['XU HUONG', 'DOANH THU PHAT HANH THE', 'VDTC', '10 NGAY']
['XU HUONG', 'TB THỰC', 'STL', '10 NGÀY']
['XU HUONG', 'TB THỰC', 'NAT', '10 NGÀY']
['XU HUONG', 'TB THỰC', 'MOV', '10 NGÀY']
['XU HUONG', 'TONG DOANH THU', 'VTS', '10 NGAY']
['XU HUONG', '1 DT BAN HANG VIEN THONG', 'VTS', '10 NGAY']
['XU HUONG', 'DICH VU SMS', 'VDS', '10 NGAY']
['XU 

## 4.4. Title Copy

In [112]:
for i in copy_list[:25]:
    print(title_list[i])

KHỐI TỈNH/TP_THUÊ BAO REGISTER_%TĂNG TRƯỞNG NGÀY CÁC ĐƠN VỊ Copy(18814)
KHỐI TỈNH/TP_THUÊ BAO DI ĐỘNG THỰC_%TĂNG TRƯỞNG NGÀY CÁC ĐƠN VỊ Copy(18816)
KHỐI TỈNH/TP_THUÊ BAO 4G THỰC_%TĂNG TRƯỞNG THÁNG CÁC ĐƠN VỊ Copy(18819)
KHỐI TỈNH/TP_THUÊ BAO FTTH_%TĂNG TRƯỞNG THÁNG CÁC ĐƠN VỊ Copy(18821)
Doanh thu (TC) Copy(18777)
VIETTEL TỈNH TP_DT dịch vụ (VTT+VTS+VDS)_TIẾN ĐỘ THÁNG CÁC ĐƠN VỊ Copy(18793)
TB REGISTER DI ĐỘNG_THEO HẠ TẦNG (LK HỆ THỐNG) Copy(18786)
VIETTEL TỈNH TP_DT dịch vụ (VTT+VTS+VDS)_TIẾN ĐỘ THÁNG CÁC ĐƠN VỊ Copy(18794)
VDTC_THẺ-TÀI KHOẢN-NẠP TIỀN_TỪNG THÁNG Copy(18767)
KHỐI TỈNH/TP_THUÊ BAO FTTH_%TĂNG TRƯỞNG NGÀY CÁC ĐƠN VỊ Copy(18820)
KHỐI TỈNH/TP_DTDV NHÓM VTT_%TĂNG TRƯỞNG NGÀY CÁC ĐƠN VỊ Copy(18800)
METFONE_THỊ PHẦN TRONG QUÝ Copy(20477)
DÒNG TIỀN RÒNG_TỪNG NĂM Copy(20527)
Top hoàn thành KH DT cộng ngang (KD) tháng Copy(4955)
CƠ CẤU TIÊU DÙNG TRẢ TRƯỚC TRẢ SAU Copy(3185) Copy(3561)
DT cộng ngang (TC) Thực hiện tháng Copy(4977)
GIANGPTH_Cơ cấu DT cộng ngang (KD) lũy kế tháng Co

In [113]:
for i in copy_list[25:50]:
    print(title_list[i])

KHỐI TỈNH/TP_THUÊ BAO 4G THỰC_%TĂNG TRƯỞNG NGÀY CÁC ĐƠN VỊ Copy(18818)
VTT_TB REGISTER DI ĐỘNG_TỪNG NGÀY Copy(15374)
Top hoàn thành KH DT cộng ngang (KD) tháng Copy(23929)
M3_DT SẢN PHẨM QUÂN SỰ_TĂNG TRƯỞNG QUÝ Copy(20713)
VTG_TB DI ĐỘNG THỰC_CƠ CẤU ĐƠN VỊ (LK KỆ THỐNG) Copy(19953)
VTS_TỔNG DT (TRỪ HUBBING)_TỪNG THÁNG Copy(19813)
VTT_TD THỰC DV DI ĐỘNG_Thừa/thiếu so với KH ngày Copy(19714)
test Copy(18490)
KHỐI TỈNH/TP_DTDV NHÓM VTT_%TĂNG TRƯỞNG THÁNG CÁC ĐƠN VỊ Copy(18801)
KHỐI TỈNH/TP_DTDV CỐ ĐỊNH_%TĂNG TRƯỞNG NGÀY CÁC ĐƠN VỊ Copy(18804)
THUÊ BAO REGISTER DI ĐỘNG Copy(24397)
KHỐI TỈNH/TP_DTDV SME_%TĂNG TRƯỞNG NGÀY CÁC ĐƠN VỊ Copy(18806)
KHỐI TỈNH/TP_DTDV SME_%TĂNG TRƯỞNG THÁNG CÁC ĐƠN VỊ Copy(18807)
KHỐI TỈNH/TP_DTDV CỐ ĐỊNH_%TĂNG TRƯỞNG THÁNG CÁC ĐƠN VỊ Copy(18805)
KHỐI TỈNH/TP_DTDV KHÁC_%TĂNG TRƯỞNG NGÀY CÁC ĐƠN VỊ Copy(18808)
Cơ cấu Doanh thu (KD) lũy kế ngày theo đơn vị Copy(20134)
KHỐI TỈNH/TP_DTDV KHÁC_%TĂNG TRƯỞNG THÁNG CÁC ĐƠN VỊ Copy(18809)
Biểu đồ cột_Mai test Copy(19621)
T

In [114]:
for i in copy_list[50:75]:
    print(title_list[i])

VDS_TỔNG DT (KD)_CƠ CẤU DỊCH VỤ Copy(20586)
Ban CNTT_TL GD HOÀN THÀNH ĐÚNG HẠN_VIETTEL_MAP Copy(22117)
VTG_TB DI ĐỘNG THỰC_CƠ CẤU ĐƠN VỊ (LK KỆ THỐNG) Copy(20822)
SXTB_TỔNG DT (TC)_KẾT QUẢ THÁNG CÁC ĐƠN VỊ Copy(23284)
TIẾN ĐỘ THỰC HIỆN NHIỆM VỤ Copy(23200)
TVTK_THU NHẬP BQ_TỪNG NĂM Copy(20579)
VTM_THU NHẬP BQ_TỪNG NĂM Copy(20581)
VHT_THU NHẬP BQ_TỪNG NĂM Copy(20582)
VIETTEL TỈNH TP_TD THỰC DV DI ĐỘNG_TĂNG TRƯỞNG THÁNG Copy(22360)
TIẾN ĐỘ THỰC HIỆN NHIỆM VỤ Copy(23241)
SXTB_TỔNG DT (TC)_TĂNG TRƯỞNG QUÝ TỪNG ĐƠN VỊ Copy(23286)
M3_THU NHẬP BQ_TỪNG NĂM Copy(20577)
XNK_THU NHẬP BQ_TỪNG NĂM Copy(20578)
HDG_DTDV_LŨY KẾ NGÀY Copy(21800)
TỈ LỆ HỆ THỐNG ĐƯỢC GIÁM SÁT CLDL Copy(24229)
Thuê bao FTTH Copy(24471)
TIẾN ĐỘ THỰC HIỆN NHIỆM VỤ_ĐẾN NGÀY /DD/MM Copy(25553)
LAO ĐỘNG BQ TRONG DS_TỪNG NĂM Copy(24804)
HOẠT ĐỘNG THU-PHÁT BƯU GỬI LŨY KẾ THÁNG Copy(25555)
TIẾN ĐỘ THỰC HIỆN NHIỆM VỤ Copy(23723)
HOẠT ĐỘNG THU-PHÁT BƯU GỬI LŨY KẾ THÁNG Copy(25551)
HCM_THỊ PHẦN THÁNG /MM/YYYY Copy(25552)
DANH SÁCH N

In [115]:
for i in copy_list[75:]:
    print(title_list[i])

Doanh thu cộng ngang (KD) Copy(25539)
TẬP ĐOÀN_DT CỘNG NGANG (KD)_KẾT QUẢ NĂM CÁC ĐƠN VỊ Copy(24425)
Doanh thu cộng ngang (KD) Copy(25275)
TB DI ĐỘNG THỰC_TỪNG NGÀY Copy(25273)
Dòng tiền về VN Copy(25373)
DOANH THU DI ĐỘNG Copy(24347)
Thuê bao di động PSC thực Copy(25386)
Thuê bao di động PSC thực Copy(25389)
Doanh thu dịch vụ Copy(25563)
DT cộng ngang (TC) Copy(25560)
Doanh thu cộng ngang (KD) Copy(25561)
DOANH THU HỢP NHẤT (TC) Copy(25562)
HOẠT ĐỘNG THU-PHÁT BƯU GỬI LŨY KẾ THÁNG Copy(25558)


## 4.5. Doanh thu cộng ngang có nhiều title

In [116]:
chart_dept[chart_dept['TITLE_CHART']=='Doanh thu cộng ngang (KD) Thực hiện tháng']

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DATE
11012,88440,CTM,19,4018,Doanh thu cộng ngang (KD) Thực hiện tháng,LINE,THANG,TỔNG DOANH THU (chi tiết),2021-11-08
11013,88441,CTM,19,4018,Doanh thu cộng ngang (KD) Thực hiện tháng,LINE_DASHED,THANG,TỔNG DOANH THU (chi tiết),2021-11-08
11602,90969,TN,19,4023,Doanh thu cộng ngang (KD) Thực hiện tháng,LINE,THANG,TỔNG DOANH THU (chi tiết),2021-07-10
11603,90970,TN,19,4023,Doanh thu cộng ngang (KD) Thực hiện tháng,LINE_DASHED,THANG,TỔNG DOANH THU (chi tiết),2021-07-10
13500,88501,VTG,19,4019,Doanh thu cộng ngang (KD) Thực hiện tháng,LINE,THANG,TỔNG DOANH THU (chi tiết),2021-10-31
13501,88502,VTG,19,4019,Doanh thu cộng ngang (KD) Thực hiện tháng,LINE_DASHED,THANG,TỔNG DOANH THU (chi tiết),2021-10-31
13565,88566,TD,19,858,Doanh thu cộng ngang (KD) Thực hiện tháng,LINE,THANG,TỔNG DOANH THU (chi tiết),2021-11-08
13566,88567,TD,19,858,Doanh thu cộng ngang (KD) Thực hiện tháng,LINE_DASHED,THANG,TỔNG DOANH THU (chi tiết),2021-11-08


In [117]:
for title in title_list:
    if 'Doanh thu cộng ngang' in title:
        print(title)

Doanh thu cộng ngang (KD) Xu hướng tháng
Doanh thu cộng ngang (KD) Thực hiện LK ngày
Doanh thu cộng ngang (KD) Xu hướng ngày
Tập đoàn_Doanh thu cộng ngang_Thừa/thiếu so với KH 6 tháng
Doanh thu cộng ngang (KD) Xu hướng quý
Doanh thu cộng ngang (KD)
Doanh thu cộng ngang (TC)
Doanh thu cộng ngang (KD) Lũy kế tháng
Doanh thu cộng ngang (TC) Thực hiện tháng
Doanh thu cộng ngang (KD) Xu hướng năm
Tập đoàn - Doanh thu cộng ngang (Kinh doanh) Copy(3184)
Doanh thu cộng ngang (KD) Thực hiện tháng
Doanh thu cộng ngang (TC) Xu hướng năm
Doanh thu cộng ngang (TC) Xu hướng tháng
Doanh thu cộng ngang (TC) Lũy kế tháng
Doanh thu cộng ngang Xu hướng tháng
Tập đoàn_Doanh thu cộng ngang_Thừa/thiếu so với KH tháng
Doanh thu cộng ngang (KD) Copy(25539)
Doanh thu cộng ngang (KD) Copy(25275)
Doanh thu cộng ngang (KD) Copy(25561)


## 4.6. Date & chu kỳ thời gian


In [118]:
chart_dept['CYCTIME'].value_counts()

THANG    31457
NGAY     15163
QUY       4605
NAM       4149
Name: CYCTIME, dtype: int64

In [119]:
chart_dept[chart_dept['DATE']=='2022-08-08'].head()

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DATE
0,77254,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,MINI,NGAY,DOANH THU DỊCH VỤ (chi tiết),2022-08-08
1,77255,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,TITLE_MOBILE,NGAY,DOANH THU DỊCH VỤ (chi tiết),2022-08-08
2,77256,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,CHART_RANK_2,NGAY,DOANH THU DỊCH VỤ (chi tiết),2022-08-08
142,77396,Q033,1148,16529,TIÊU DÙNG THỰC DI ĐỘNG,DYNAMIC_TEXT,NGAY,Tiêu dùng thực di động (TT+TS),2022-08-08
143,77397,Q033,1148,16529,TIÊU DÙNG THỰC DI ĐỘNG,BAR_ALARM,NGAY,Tiêu dùng thực di động (TT+TS),2022-08-08


## 4.7. Dept_code


In [120]:
# 185 dept_code
check_list = chart_dept['DEPT_CODE'].apply(lambda x: x.lower().strip())
list_dept_code = list(set(check_list))
len(list_dept_code)

167

In [121]:
list_dept_code

['v064',
 'bncsx',
 'myn',
 'vdtc',
 'h031',
 'n038',
 'cpltm',
 'h019',
 'b241',
 'b650',
 't079',
 'btt',
 'vttnewnew',
 'bct',
 'tn',
 'd230',
 't073',
 'xnkvg',
 'h320',
 'l025',
 'q033',
 'mov',
 'bkt',
 'vta',
 'c710',
 'idc',
 'hc',
 'anm',
 'vtb',
 'dtk',
 'bc',
 'btckt',
 'vds',
 'q510',
 'b062',
 'd501',
 'k077',
 'vts',
 'vtz',
 'c026',
 'sxtb',
 'bhc',
 'm1',
 'b240',
 'b281',
 'a076',
 'sxtbcnc',
 't037',
 'xmcp',
 'mvt',
 'h018',
 'kgm',
 't054',
 'h004',
 'vdb',
 'vtnnhn',
 'h321',
 'vtk',
 'vcm',
 'vmc',
 'tvtk',
 'vttnewew',
 'vtnn',
 'xmcpv',
 'k060',
 'hvt',
 'vtcm',
 'q003',
 'd500',
 'ctm',
 'v070',
 'vcr',
 'l072',
 'y029',
 'b056',
 't008',
 'vte',
 'dvk',
 'bdttc',
 'btcnl',
 'đvđl',
 'q052',
 'l020',
 'vttnew',
 'th',
 'vtghn',
 'b075',
 'b781',
 'tt',
 'q053',
 'vht',
 'vtl',
 'cntt',
 'bdn',
 'vtc',
 'td',
 'vtn',
 'nat',
 'bdtxd',
 "bc'",
 't280',
 'tttt',
 'l231',
 'vgo',
 'bcnc',
 't027',
 'n350',
 'vp',
 'vbd',
 'dstct',
 'p057',
 'm3',
 'tv360',
 'bdvdc'

## 4.8. SERVICE_ID

In [122]:
# 1173 service id
list_serviceid = list(set(chart_dept['SERVICE_ID']))
len(list_serviceid)
chart_dept[chart_dept['SERVICE_ID']==list_serviceid[5]]

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DATE
19496,86231,TD,5,1976,Tiêu dùng Lũy kế ngày,LINE,NGAY,TD GỐC DI ĐỘNG TT,2021-03-11


In [126]:
len(chart_dept['TYPE_CHART'].value_counts())

47

## 4.9. Type Chart

In [123]:
chart_dept['TYPE_CHART'].value_counts()[:20]

COLUMN                   13943
LINE                     11728
SHARE_TIME                3555
LINE_DASHED               3210
MINI                      2762
BAR_ALARM                 2666
DYNAMIC_TEXT              2514
TITLE_MOBILE              2463
PIE                       2271
TABLE_CONTENT             2021
CHART_RANK_2              1388
PERFORMANCE_CHART_2       1253
ITEM_BLOCK_INFO_2_KPI      975
BELOW_COLUMN               879
OVER_COLUMN                878
ITEM_NESTED_PIE            739
MINI_ALARM2                465
TEXT_KPI                   412
PERFORMANCE_CHART          410
INFO_MAIN                  397
Name: TYPE_CHART, dtype: int64

In [124]:
chart_dept['TYPE_CHART'].value_counts()[20:]

TEXT_DELTA_YEAR           381
TOTAL                     299
DIFFERENCE                298
T_COLUMN                  295
TEXT_PLAN_MONTH           268
STACK                     156
TEXT_PLAN_DAY             142
GROUP_BAR                 134
1                         131
BAR                       107
DYNAMIC_TEXT_2             94
Chart INFO_OBJ             21
TABLE_BAR                  18
TEXT_DELTA_MONTH           15
TEXT_DELTA_DAY             12
STACK_BARPLOT              10
RADAR                       5
TABLE_CONTENT_THI_PHAN      4
LAST_VAL_BLOCK              2
TABLE                       2
VAL_BLOCK                   2
LAST_VAL_YEAR               2
PLAN_BLOCK                  2
ITEM                        2
AREA                        1
RANK_GOOD                   1
RANK_BAD                    1
Name: TYPE_CHART, dtype: int64

In [133]:
check = ['TABLE_CONTENT', 'STACK_BARPLOT', 'ITEM_NESTED_PIE', 'LINE', 'LINE_DASHED', 'MINI_ALARM2', 'MINI', 'Chart INFO_OBJ', 'TITLE_MOBILE', 'PIE', 'BELOW_COLUMN', 'AREA', 'RADAR', 'SHARE_TIME', 'COLUMN', 'BAR', 'ITEM_BLOCK_INFO_2_KPI', 'INFO_MAIN', 'T_COLUMN', 'STACK', 'GROUP_BAR', 'PERFORMANCE_CHART_2', '1', 'DYNAMIC_TEXT', 'OVER_COLUMN']
check = [i for i in chart_dept['TYPE_CHART'].unique() if i not in check]
check

['CHART_RANK_2',
 'BAR_ALARM',
 'TOTAL',
 'DIFFERENCE',
 'TEXT_DELTA_YEAR',
 'TEXT_KPI',
 'PERFORMANCE_CHART',
 'TEXT_PLAN_DAY',
 'TEXT_PLAN_MONTH',
 'TEXT_DELTA_DAY',
 'TEXT_DELTA_MONTH',
 'PLAN_BLOCK',
 'LAST_VAL_YEAR',
 'VAL_BLOCK',
 'LAST_VAL_BLOCK',
 'TABLE',
 'RANK_GOOD',
 'RANK_BAD',
 'DYNAMIC_TEXT_2',
 'ITEM',
 'TABLE_CONTENT_THI_PHAN',
 'TABLE_BAR']

In [134]:
chart_dept[chart_dept['TYPE_CHART']=='TABLE_CONTENT_THI_PHAN']

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DATE
56002,132862,VTTNEWNEW,20212743,22584,Viettel Tỉnh/TP_Diễn biến thị phần,TABLE_CONTENT_THI_PHAN,THANG,Viettel 2743,2023-06-20
56153,133821,H004,20212743,25721,HNI_Diễn biến thị phần,TABLE_CONTENT_THI_PHAN,THANG,Viettel 2743,2023-06-29
56159,132821,VTG,1200700,25579,THỊ PHẦN THÁNG /MM/YYYY,TABLE_CONTENT_THI_PHAN,THANG,Smart,2023-06-02
56342,133175,VTTNEWNEW,20212743,25584,VTT_Diễn biến thị phần_từng tỉnh,TABLE_CONTENT_THI_PHAN,THANG,Viettel 2743,2023-06-20


In [40]:
type_chart_list = list(set(chart_dept['TYPE_CHART']))
type_chart_list[:30]

['INFO_MAIN',
 'PLAN_BLOCK',
 'TEXT_DELTA_YEAR',
 'GROUP_BAR',
 'T_COLUMN',
 'LINE',
 'TEXT_PLAN_DAY',
 '1',
 'AREA',
 'ITEM_NESTED_PIE',
 'TABLE',
 'DYNAMIC_TEXT_2',
 'STACK',
 'LAST_VAL_BLOCK',
 'RADAR',
 'CHART_RANK_2',
 'DYNAMIC_TEXT',
 'TABLE_CONTENT',
 'VAL_BLOCK',
 'ITEM',
 'TABLE_BAR',
 'LAST_VAL_YEAR',
 'SHARE_TIME',
 'RANK_BAD',
 'BAR',
 'TEXT_DELTA_DAY',
 'RANK_GOOD',
 'PIE',
 'MINI_ALARM2',
 'COLUMN']

In [41]:
type_chart_list[30:]

['BAR_ALARM',
 'PERFORMANCE_CHART',
 'TITLE_MOBILE',
 'DIFFERENCE',
 'MINI',
 'STACK_BARPLOT',
 'TABLE_CONTENT_THI_PHAN',
 'TEXT_KPI',
 'BELOW_COLUMN',
 'LINE_DASHED',
 'ITEM_BLOCK_INFO_2_KPI',
 'TEXT_PLAN_MONTH',
 'TEXT_DELTA_MONTH',
 'PERFORMANCE_CHART_2',
 'OVER_COLUMN',
 'Chart INFO_OBJ',
 'TOTAL']

In [42]:
cat_item

,ITEM_ID,ITEM_CODE,ITEM_NAME,ITEM_VALUE,POSITION,DESCRIPTION,STATUS
0,990,TYPE_CHART_TABLE,Biểu đồ TABLE,TABLE,17.0,TABLE,1
1,918,TYPE_CHART_LINE_DASHED,LINE_DASHED chart,LINE_DASHED,17.0,Biểu đồ LINE_DASHED,0
2,306083,TYPE_CHART_ALARM_CHART_2,Biểu đồ cảnh báo 2,ALARM_CHART_2,21.0,Biểu đồ cảnh báo 2,1
3,915,TYPE_CHART_INFO_MAIN,Biểu đồ INFO_MAIN,INFO_MAIN,14.0,INFO_MAIN,1
4,916,TYPE_CHART_SHARE_TIME,Biểu đồ cơ cấu cột chồng,SHARE_TIME,15.0,SHARE_TIME,1
5,917,TYPE_CHART_RANK,Biểu đồ RANK,RANK,16.0,RANK,1
6,304828,TYPE_CHART_NESTED_PIE,Biểu đồ NESTED_PIE,NESTED_PIE,20.0,Biểu đồ NESTED_PIE,1
7,304802,TYPE_CHART_BLOCK_INFO,Biểu đồ BLOCK_INFO,BLOCK_INFO,19.0,Biểu đồ BLOCK_INFO,1
8,305027,BLOCK_INFO_2,Biểu đồ BLOCK_INFO_2,BLOCK_INFO_2,19.0,Biểu đồ BLOCK_INFO_2,1
9,914,TYPE_CHART_INFO_OBJ,Biểu đồ INFO_OBJ,INFO_OBJ,13.0,INFO_OBJ,1


# 5. Dashboard Param Tree

3 columns 1 biến: "STATUS", "TYPE_PARAM", "GROUP_TYPE" \
6 columns Nan full: "PARAM_ORDER_SMS", "DEPT_ORDER", "SUB_CODE", "IMAGE", "START_TIME", "END_TIME" \
1 column do not need: "MODIFIED_DATE"

CODE - NAME: song ánh
Parent: Tỉnh (Example: T027: Tuyên Quang)

In [43]:
# Dashboard param tree
dashboard_param = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/DASHBOARD_PARAM_TREE_202307280939.csv")
dashboard_param.drop(columns=["PARAM_ORDER_SMS", "DEPT_ORDER", "SUB_CODE", "IMAGE",
                              "STATUS", "TYPE_PARAM", "GROUP_TYPE",
                              "START_TIME", "END_TIME",
                              "MODIFIED_DATE"], inplace=True)

In [44]:
dashboard_param

,CODE,NAME,PARENT,PARAM_ORDER,LEVEL_NODE,ID,DEPT_ID,PARENT_DEPT_ID,STANDARD_NAME
0,VTG_METFONE,"Metfone_CT Viettel Cambodia Pte., Ltd",VTG,0,11.0,100676,308469,256.0,NaN
1,TTTT,VTSport_TT Thể thao Viettel,CPLTM,5,NaN,100698,2108,308480.0,NaN
2,HVT,VTAca_Học viện Viettel,CPLTM,4,NaN,100699,305965,308480.0,NaN
3,VTH,TT KD Sản phẩm công nghệ cao Viettel,SXTBCNC_HHL,1,NaN,100724,7920,308482.0,NaN
4,VTEK,TT Nghiên cứu công nghệ mạng Viettel,SXTBCNC_HHL,0,NaN,100725,1709,308482.0,NaN
...,...,...,...,...,...,...,...,...,...
838,Y029006,H.Trấn Yên,Y029,4,NaN,101613,949,314.0,H.Trấn Yên
839,Y029008,H.Văn Chấn,Y029,5,NaN,101614,951,314.0,H.Văn Chấn
840,Y029004,H.Văn Yên,Y029,6,NaN,101615,947,314.0,H.Văn Yên
841,Y029007,H.Yên Bình,Y029,7,NaN,101616,950,314.0,H.Yên Bình


In [45]:
dashboard_param['DEPT_ID'].unique

<bound method Series.unique of 0      308469
1        2108
2      305965
3        7920
4        1709
        ...  
838       949
839       951
840       947
841       950
842       945
Name: DEPT_ID, Length: 843, dtype: int64>

In [46]:
parent_list = list(set(dashboard_param['PARENT']))
len(parent_list)

74

In [47]:
code_list = list(set(dashboard_param['CODE']))
len(code_list)

843

In [48]:
deptid_list = list(set(dashboard_param['DEPT_ID']))
len(deptid_list)

842

In [49]:
parentdeptid_list = list(set(dashboard_param['PARENT_DEPT_ID']))
len(parentdeptid_list)

78

In [50]:
dashboard_param[dashboard_param['PARENT']=='VTG']

,CODE,NAME,PARENT,PARAM_ORDER,LEVEL_NODE,ID,DEPT_ID,PARENT_DEPT_ID,STANDARD_NAME
0,VTG_METFONE,"Metfone_CT Viettel Cambodia Pte., Ltd",VTG,0,11.0,100676,308469,256.0,NaN
30,VTCM,Nexttel_CT Viettel Cameroon S.A.R.L,VTG,9,NaN,100681,308341,256.0,NaN
31,VTG_BITEL,Bitel_CT Viettel Peru,VTG,8,NaN,100682,308478,256.0,Bitel
32,VTG_LUMITEL,Lumitel_CT Viettel Burundi S.A,VTG,7,NaN,100683,308476,256.0,NaN
33,VTG_HALOTEL,Halotel_CT Viettel Tanzania,VTG,6,NaN,100684,308475,256.0,NaN
34,VTG_MOVITEL,Movitel_CT Movitel,VTG,5,NaN,100685,308474,256.0,NaN
89,VTG_NATCOM,"Natcom_CT Natcom, S.A",VTG,4,NaN,100677,308473,256.0,NaN
90,VTG_MYTEL,"Mytel_Telecom International Myanmar Co., Ltd",VTG,3,NaN,100678,308472,256.0,NaN
91,VTG_TELEMOR,Telemor_CT Viettel Timor Leste,VTG,2,NaN,100679,308471,256.0,NaN
92,VTG_UNITEL,Unitel_CT Star Telecom,VTG,1,NaN,100680,308470,256.0,NaN


# 6. Analyze

## 6.1. Check đơn vị in chart_title (Fail)

In [51]:
import numpy as np
donvi_list = list(set(dashboard_param['PARENT']))
donvi_list.remove(np.nan)
donvi_list = [i.lower() for i in donvi_list]
donvi_list[:5]

['y029', 'q052', 'c026', 'd511', 'l072']

In [52]:
title_chart_list = list(set(chart_dept['TITLE_CHART']))
title_chart_list = [i.lower() for i in title_chart_list]
title_chart_list[:5]

['vht_doanh thu_từng tháng',
 'bdh_dtdv nhóm vds_từng tháng',
 'nan_tb di động thực_từng năm',
 'hyn_tb ftth_từng quý',
 'tổng pa về dv viettelpay / tổng tb_từng tháng']

In [53]:
alltitle = []
for title in title_chart_list:
    eachtitle = []
    for donvi in donvi_list:
        if donvi in title:
            eachtitle.append(donvi)
    alltitle.append(eachtitle)
    

In [54]:
i = 4
alltitle[20*i:20*(i+1)]

[[],
 ['td', 'vt'],
 ['vt'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['td', 'vt'],
 [],
 [],
 [],
 [],
 ['td'],
 [],
 [],
 [],
 [],
 []]

In [55]:
sum([len(i)>0 for i in alltitle])

3594

## 6.2. Sentence Transformer trích xuất thời gian

In [56]:
title_chart_list = list(set(chart_dept['TITLE_CHART']))
title_chart_list = [i.lower() for i in title_chart_list]
title_chart_list[:5]

['vht_doanh thu_từng tháng',
 'bdh_dtdv nhóm vds_từng tháng',
 'nan_tb di động thực_từng năm',
 'hyn_tb ftth_từng quý',
 'tổng pa về dv viettelpay / tổng tb_từng tháng']

In [57]:
chart_dept[chart_dept['SERVICE_ID']==143].head()

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DATE
0,77254,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,MINI,NGAY,DOANH THU DỊCH VỤ (chi tiết),2022-08-08
1,77255,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,TITLE_MOBILE,NGAY,DOANH THU DỊCH VỤ (chi tiết),2022-08-08
2,77256,B281,143,16558,DOANH THU DỊCH VỤ TỔNG,CHART_RANK_2,NGAY,DOANH THU DỊCH VỤ (chi tiết),2022-08-08
26,77280,G059,143,13169,GLI_DTDV_TỪNG QUÝ,LINE,QUY,DOANH THU DỊCH VỤ (chi tiết),2022-06-03
32,77286,H018,143,12866,HBH_DTDV_TỪNG QUÝ,LINE,QUY,DOANH THU DỊCH VỤ (chi tiết),2021-10-26


In [58]:
def remove_donvi(title_chart):
    features = title_chart.split("_")
    if " " not in features[0] and features[0] not in ["company", 'info', 'test', 'rank', 'rada', 'table', 'tooltip', '21', 'dgsdf', 'khdn', 'dt-cp-ln', 'arpu', 'mou', 'dou']:
        output =  features[0]
        output2 = "_".join(features[1:])
    elif features[0] in ['tập đoàn', 'ban cntt', 'khối tỉnh/tp', 'công đoàn', 'viettel tỉnh/tp']:
        output = features[0]
        output2 = "_".join(features[1:])
    else:
        output = ""
        output2 = "_".join(features[0:])
    return output, output2
    

def remove_time(title_chart_rm_donvi):
    features = title_chart_rm_donvi.split("_")
    if len(features) == 1:
        features = "_".join(features)
        return features, ""
    output2 = ""
    output = deepcopy(features)
    for feature in features:
        for i in ['ngay', 'thang', 'nam', 'quy', 'ngày', 'tháng', 'năm', 'quý']:
            if i in feature:
                output2 = feature
                output.remove(feature)
                break
    
    output = "_".join(output)
    return output, output2

In [59]:
chart_dept_for_title = deepcopy(chart_dept)
chart_dept_for_title.drop(columns=['DATE'], inplace=True)
chart_dept_for_title = chart_dept_for_title.apply(lambda col: col.str.lower() if col.dtype!='int64' else col)
chart_dept_for_title['DONVI_FEATURE'] = chart_dept_for_title['TITLE_CHART'].apply(lambda x: remove_donvi(x)[0])
chart_dept_for_title['TITLE_CHART_RM_DONVI'] = chart_dept_for_title['TITLE_CHART'].apply(lambda x: remove_donvi(x)[1])
chart_dept_for_title['TIME_FEATURE'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[1])
chart_dept_for_title['TITLE_CHART_RM_DONVI_TIME'] = chart_dept_for_title['TITLE_CHART_RM_DONVI'].apply(lambda x: remove_time(x)[0])
chart_dept_for_title.drop(columns=['TITLE_CHART_RM_DONVI'], inplace=True)

In [60]:
i = 1400
chart_dept_for_title[20*i:20*(i+1)]

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DONVI_FEATURE,TIME_FEATURE,TITLE_CHART_RM_DONVI_TIME
28000,104389,k058,20214133,9291,kha_dtdv nhóm vtt_lũy kế ngày,line,ngay,dt dịch vụ nhóm dv của vtt,kha,lũy kế ngày,dtdv nhóm vtt
28001,104390,k058,20214133,9292,kha_dtdv nhóm vtt_từng tháng,line,thang,dt dịch vụ nhóm dv của vtt,kha,từng tháng,dtdv nhóm vtt
28002,104391,v211,143,15641,doanh thu dịch vụ tổng,dynamic_text,ngay,doanh thu dịch vụ (chi tiết),,,doanh thu dịch vụ tổng
28003,104392,v211,143,15641,doanh thu dịch vụ tổng,bar_alarm,ngay,doanh thu dịch vụ (chi tiết),,,doanh thu dịch vụ tổng
28004,104393,v211,143,15641,doanh thu dịch vụ tổng,mini,ngay,doanh thu dịch vụ (chi tiết),,,doanh thu dịch vụ tổng
28005,104394,v211,143,15641,doanh thu dịch vụ tổng,title_mobile,ngay,doanh thu dịch vụ (chi tiết),,,doanh thu dịch vụ tổng
28006,104395,v211,143,15641,doanh thu dịch vụ tổng,chart_rank_2,ngay,doanh thu dịch vụ (chi tiết),,,doanh thu dịch vụ tổng
28007,104396,b240,275,9253,bgg_tb ftth_từng ngày,line,ngay,fttx - tb thực lũy kế,bgg,từng ngày,tb ftth
28008,104397,b240,276,9253,bgg_tb ftth_từng ngày,column,ngay,fttx - tb thực tăng/giảm,bgg,từng ngày,tb ftth
28009,104398,h031,20214133,9380,hpg_dtdv nhóm vtt_từng tháng,line,thang,dt dịch vụ nhóm dv của vtt,hpg,từng tháng,dtdv nhóm vtt


In [61]:
chart_dept_for_title

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DONVI_FEATURE,TIME_FEATURE,TITLE_CHART_RM_DONVI_TIME
0,77254,b281,143,16558,doanh thu dịch vụ tổng,mini,ngay,doanh thu dịch vụ (chi tiết),,,doanh thu dịch vụ tổng
1,77255,b281,143,16558,doanh thu dịch vụ tổng,title_mobile,ngay,doanh thu dịch vụ (chi tiết),,,doanh thu dịch vụ tổng
2,77256,b281,143,16558,doanh thu dịch vụ tổng,chart_rank_2,ngay,doanh thu dịch vụ (chi tiết),,,doanh thu dịch vụ tổng
3,77257,s079,970,12386,stg_tb register di động_theo hạ tầng (lk hệ th...,line,thang,4g lũy kế,stg,,tb register di động_theo hạ tầng (lk hệ thống)
4,77258,s079,971,12386,stg_tb register di động_theo hạ tầng (lk hệ th...,line,thang,3g tăng giảm,stg,,tb register di động_theo hạ tầng (lk hệ thống)
...,...,...,...,...,...,...,...,...,...,...,...
57329,132568,td,6003,23456,danh sách nhiệm vụ chậm tiến độ,table_content,NaN,chậm tiến độ,,,danh sách nhiệm vụ chậm tiến độ
57330,132353,td,6002,23457,danh sách sắp đến hạn,table_content,NaN,sắp đến hạn,,,danh sách sắp đến hạn
57331,132354,td,6003,23456,danh sách nhiệm vụ chậm tiến độ,table_content,NaN,chậm tiến độ,,,danh sách nhiệm vụ chậm tiến độ
57332,132355,td,6002,23457,danh sách sắp đến hạn,table_content,NaN,sắp đến hạn,,,danh sách sắp đến hạn


In [62]:
chart_dept_for_title['TYPE_CHART'].unique()

array(['mini', 'title_mobile', 'chart_rank_2', 'line', 'dynamic_text',
       'bar_alarm', 'column', 'share_time', 'line_dashed', 'pie',
       'performance_chart_2', 'over_column', 'below_column', 'total',
       'difference', 't_column', 'mini_alarm2', 'info_main',
       'text_delta_year', 'text_kpi', 'performance_chart',
       'text_plan_day', 'item_block_info_2_kpi', 'text_plan_month',
       'item_nested_pie', 'stack_barplot', 'text_delta_day',
       'text_delta_month', 'plan_block', 'last_val_year', 'val_block',
       'last_val_block', 'stack', 'area', 'bar', 'table',
       'chart info_obj', 'radar', 'group_bar', 'table_content', '1',
       'rank_good', 'rank_bad', 'dynamic_text_2', 'item',
       'table_content_thi_phan', 'table_bar'], dtype=object)

In [63]:
chart_dept_for_title[chart_dept_for_title['CHART_ID']==14395]

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DONVI_FEATURE,TIME_FEATURE,TITLE_CHART_RM_DONVI_TIME
21879,101293,vttnewnew,19,14395,tổng doanh thu (kd),performance_chart_2,thang,tổng doanh thu (chi tiết),,,tổng doanh thu (kd)
21880,101294,vttnewnew,19,14395,tổng doanh thu (kd),dynamic_text,thang,tổng doanh thu (chi tiết),,,tổng doanh thu (kd)
21881,101295,vttnewnew,19,14395,tổng doanh thu (kd),bar_alarm,thang,tổng doanh thu (chi tiết),,,tổng doanh thu (kd)
21882,101296,vttnewnew,19,14395,tổng doanh thu (kd),mini_alarm2,thang,tổng doanh thu (chi tiết),,,tổng doanh thu (kd)
21883,101297,vttnewnew,19,14395,tổng doanh thu (kd),mini,thang,tổng doanh thu (chi tiết),,,tổng doanh thu (kd)
21884,101298,vttnewnew,19,14395,tổng doanh thu (kd),title_mobile,thang,tổng doanh thu (chi tiết),,,tổng doanh thu (kd)


In [64]:
chart_dept_for_title[(chart_dept_for_title['TYPE_CHART']=='mini_alarm2') & (chart_dept_for_title['DEPT_CODE']=='vttnewnew') & (chart_dept_for_title['SERVICE_ID']==19) & (chart_dept_for_title['CYCTIME']=='thang')]

,ID,DEPT_CODE,SERVICE_ID,CHART_ID,TITLE_CHART,TYPE_CHART,CYCTIME,SERVICE_NAME,DONVI_FEATURE,TIME_FEATURE,TITLE_CHART_RM_DONVI_TIME
21882,101296,vttnewnew,19,14395,tổng doanh thu (kd),mini_alarm2,thang,tổng doanh thu (chi tiết),,,tổng doanh thu (kd)
42734,120881,vttnewnew,19,24035,doanh thu cộng ngang (kd),mini_alarm2,thang,tổng doanh thu (chi tiết),,,doanh thu cộng ngang (kd)
54328,128864,vttnewnew,19,24453,doanh thu cộng ngang (kd),mini_alarm2,thang,tổng doanh thu (chi tiết),,,doanh thu cộng ngang (kd)
56451,132805,vttnewnew,19,25565,tổng doanh thu (kd),mini_alarm2,thang,tổng doanh thu (chi tiết),,,tổng doanh thu (kd)


In [65]:
chart_dept_for_title['DEPT_CODE'].unique()

array(['b281', 's079', 'g059', 'd061', 'q053', 't073', 'h018', 'd511',
       'p210', 'q510', 'p057', 'vttnewnew', 'stl', 'h320', 'd500',
       'sxtbcnc', 't036', 'nat', 'q033', 'd501', 'td', 'b651', 'hvt',
       'vds', 'xnk', 'vtp', 'myn', 'th', 'mov', 'vnc', 'vhk', 'idc', 'm3',
       'vdtc', 'vtc', 'vtz', 'vtl', 'vbd', 'xmcp', 'm1', 'bc', 'ct',
       'tvtk', 'vtg', 'b241', 'anm', 'q055', 'kgm', 'vts', 's022', 'vta',
       'vte', 'vtr', 'q052', 'c710', 'h039', 'vp_g', 'tn', 'sxtb', 'cntt',
       'ktm', 'kxl', 'dvk', 'dtk', 'vtn', 'xnkvg', 'vht', 'bds', 'ctm',
       'vtx', 'hc', 'b781', 'n350', 'd230', 'h031', 'l025', 'n030',
       'l063', 't054', 'n351', 'd067', 'n068', 't008', 'h019', 'c780',
       'l072', 'b062', 'y029', 'l231', 'n038', 'c026', 'k060', 'v064',
       'vtt', 'h004', 'l020', 'b650', 'b240', 'v211', 'h321', 'b075',
       'vtcm', 'v070', 'k058', 'b056', '  stl', '  nat', '  mov', '  vtl',
       '  vtp', '  vcr', '  vbd', '  vtz', '  myn', 't027', 't280',
    

In [66]:
dashboard_param = pd.read_csv("/Users/tranxuanhuy/Documents/Viettel Digital Talent/Chatbot/Code/data/DASHBOARD_PARAM_TREE_202307280939.csv")
dashboard_param.drop(columns=["PARAM_ORDER_SMS", "DEPT_ORDER", "SUB_CODE", "IMAGE",
                              "STATUS", "TYPE_PARAM", "GROUP_TYPE",
                              "START_TIME", "END_TIME",
                              "MODIFIED_DATE"], inplace=True)
dashboard_param.

SyntaxError: invalid syntax (<ipython-input-66-45457225358b>, line 6)